# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import gmaps

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,ilulissat,69.2167,-51.1000,266.16,73,75,0.00,GL,1701670075
1,1,sabang,5.8933,95.3214,300.69,75,99,2.07,ID,1701670075
2,2,bandon,51.7469,-8.7425,273.92,89,51,8.86,IE,1701670076
3,3,port-aux-francais,-49.3500,70.2167,277.13,71,15,14.75,TF,1701670076
4,4,kodiak,57.7900,-152.4072,275.88,81,100,4.12,US,1701670076


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
gmaps.configure(api_key=geoapify_key)

# Display the map
map_plot = city_data_df.hvplot.points('Lng',
                                    'Lat',
                                    geo=True,
                                    size='Humidity',
                                    scale=1,
                                    color='City',
                                    tiles='OSM',
                                    )

map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
temp_range = (city_data_df["Max Temp"] >= 250) & (city_data_df["Max Temp"] <= 350)
humidity_range = (city_data_df["Humidity"] >= 20) & (city_data_df["Humidity"] <= 50) 
wind_threshold = city_data_df["Wind Speed"] <= 15
cloudiness_threshold = city_data_df["Cloudiness"] <= 30

ideal_df = city_data_df[temp_range & humidity_range & wind_threshold & cloudiness_threshold].copy()

# Drop any rows with null values
ideal_df.dropna(inplace=True)


# Display sample data
ideal_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
16,16,toyooka,35.5333,134.8333,285.69,49,18,0.89,JP,1701670078
31,31,san martin de los andes,-40.1604,-71.3486,278.00,39,0,0.71,AR,1701670088
82,82,linxia chengguanzhen,35.6003,103.2064,280.66,26,0,1.07,CN,1701670104
85,85,ormara,25.2088,64.6357,299.85,30,4,5.31,PK,1701670105
89,89,merowe,18.4833,31.8167,297.86,35,7,5.40,SD,1701670106


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()


# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""


# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
16,toyooka,JP,35.5333,134.8333,49,
31,san martin de los andes,AR,-40.1604,-71.3486,39,
82,linxia chengguanzhen,CN,35.6003,103.2064,26,
85,ormara,PK,25.2088,64.6357,30,
89,merowe,SD,18.4833,31.8167,35,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
params = {'categories': "accommodation.hotel",
          'apiKey': geoapify_key,
         }

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row['Lat']
    lng = row['Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
toyooka - nearest hotel: No hotel found
san martin de los andes - nearest hotel: No hotel found
linxia chengguanzhen - nearest hotel: No hotel found
ormara - nearest hotel: No hotel found
merowe - nearest hotel: No hotel found
nguigmi - nearest hotel: No hotel found
illizi - nearest hotel: No hotel found
annau - nearest hotel: No hotel found
buka - nearest hotel: No hotel found
yazd - nearest hotel: No hotel found
yima - nearest hotel: No hotel found
kavalerovo - nearest hotel: No hotel found
crane - nearest hotel: No hotel found
penhalonga - nearest hotel: No hotel found
quelimane - nearest hotel: No hotel found
lamesa - nearest hotel: No hotel found
emerald - nearest hotel: No hotel found
hami - nearest hotel: No hotel found
gray - nearest hotel: No hotel found
i-n-salah - nearest hotel: No hotel found
qalat - nearest hotel: No hotel found
moranbah - nearest hotel: No hotel found
farsan - nearest hotel: No hotel found
tonghae - nearest hotel: No hotel found
hasa

,City,Country,Lat,Lng,Humidity,Hotel Name
16,toyooka,JP,35.5333,134.8333,49,No hotel found
31,san martin de los andes,AR,-40.1604,-71.3486,39,No hotel found
82,linxia chengguanzhen,CN,35.6003,103.2064,26,No hotel found
85,ormara,PK,25.2088,64.6357,30,No hotel found
89,merowe,SD,18.4833,31.8167,35,No hotel found
93,nguigmi,NE,14.2495,13.1092,24,No hotel found
138,illizi,DZ,26.4833,8.4667,46,No hotel found
172,annau,TM,37.8876,58.5160,21,No hotel found
173,buka,UZ,40.8108,69.1986,30,No hotel found
186,yazd,IR,31.8972,54.3675,24,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
hotels_plot = hotel_df.hvplot.points(
    "Lng", 
    "Lat", 
    geo = True, 
    size = "Humidity",
    scale = 1,
    color = "City",
    tiles = "OSM",
    hover_cols=list(hotel_df.columns)
    
)

# Display the map
hotels_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Hotel Name)